In [7]:
#!g1.1
from torch.profiler import profile, record_function, ProfilerActivity
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import grad, Variable
from torchvision.utils import save_image
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torch

from IPython.display import clear_output, HTML, display
from matplotlib import animation, pyplot as plt
from time import gmtime, strftime
from tqdm import tqdm
from PIL import Image
import numpy as np
import wandb
import os

In [11]:
#!g1.1
!rm -rf 'Weight/StyleGAN R1 64'

In [12]:
#!g1.1
from start import init_train

Trainer = init_train("configs/StyleGAN.json", True)

Generator(
  (mapping): Mapping(
    (blocks): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (2): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (3): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (4): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (5): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (6): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1):

 41%|████      | 37987/92219 [03:21<04:20, 208.57it/s]

Error: connection with executor has been lost. Please try again.

In [16]:
#!g1.1
!cp trainer.py starter.ipynb start.py losses.py -r utils models configs Project/
!tar -cvf project.tar Project/

Project/
Project/trainer.py
Project/starter.ipynb
Project/start.py
Project/losses.py
Project/utils/
Project/utils/.ipynb_checkpoints/
Project/utils/register.py
Project/utils/__pycache__/
Project/utils/__pycache__/utils.cpython-37.pyc
Project/utils/__pycache__/register.cpython-37.pyc
Project/utils/__pycache__/images.cpython-37.pyc
Project/utils/images.py
Project/models/
Project/models/StyleGAN.py
Project/models/.ipynb_checkpoints/
Project/models/__pycache__/
Project/models/__pycache__/R1GAN.cpython-37.pyc
Project/models/__pycache__/StyleGAN_fix.cpython-37.pyc
Project/models/__pycache__/StyleGAN.cpython-37.pyc
Project/models/R1GAN.py
Project/models/StyleGAN_fix.py
Project/configs/
Project/configs/StyleGAN.json
Project/configs/.ipynb_checkpoints/
Project/configs/R1GAN.json


In [ ]:
#!g1.1
Trainer.train_loop()

In [ ]:
#!g1.1
next(iter(Trainer.dataloader))[0].shape

In [ ]:
#!g1.1
from utils.images import TensorToImage
random_img = lambda: int(np.random.rand(1)[0] * len(dataset))
plot_width, plot_height = 10, 5

fig = plt.figure(figsize=(3 * plot_width, 3 * plot_height))
for sample_n in range(1, plot_width * plot_height + 1):
    ax = fig.add_subplot(plot_height, plot_width, sample_n)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.imshow(TensorToImage(next(iter(Trainer.dataloader))[sample_n].detach().cpu(), 0.5, 0.375))

In [ ]:
#!g1.1
state = torch.load('Weight/StyleGAN StyleGAN 64/weight 62.pth')

In [ ]:
#!g1.1
state['D'].pop('fromRGB.weight', None)
state['D'].pop('fromRGB.bias', None)
state['D'].pop('Linear.weight', None)

In [ ]:
#!g1.1
Trainer.D.load_state_dict(state['D'], strict=False)

In [ ]:
#!g1.1
state['G'].pop('to_rgb.weight', None)

In [ ]:
#!g1.1
Trainer.G.load_state_dict(state['G'], strict=False)

In [ ]:
#!g1.1
Trainer.train_loop()

In [ ]:
#!g1.1
next(iter(Trainer.dataloader))[0].shape

In [ ]:
#!g1.1
from utils.images import TensorToImage

random_img = lambda: int(np.random.rand(1)[0] * len(dataset))
plot_width, plot_height = 10, 10

fig = plt.figure(figsize=(3 * plot_width, 3 * plot_height))
for sample_n in range(1, plot_width * plot_height + 1):
    ax = fig.add_subplot(plot_height, plot_width, sample_n)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.imshow(TensorToImage(Trainer.generate_images().view(3, 128, 128).detach().cpu(), 0.5, 0.375))

In [ ]:
#!g1.1
plt.imshow(TensorToImage(Trainer.generate_images().view(3, 128, 128).detach().cpu(), 0.5, 0.375))

In [ ]:
#!g1.1
w = Trainer.G.mapping(torch.randn((1, 128), device=Trainer.device))
img = Trainer.G.const.expand(w.size(0), -1, -1, -1)

In [ ]:
#!g1.1
len(Trainer.G.blocks)

In [ ]:
#!g1.1
img2 = Trainer.G.blocks[0](img, w)
plt.imshow(TensorToImage(img2.view(3, 4, 4).detach().cpu(), 0.5, 0.375))

In [ ]:
#!g1.1
!python3 start.py "configs/StyleGAN.json" True

In [ ]:
#!g1.1
def generate_video(samples_list, delay=1000):
    def process_image(img):
        if len(img.shape) == 4:
            img = img[0]
        plt.axis('off')
        return [plt.imshow(TensorToImage(img, 0.5, 0.28))]

    fig = plt.figure(figsize=(8, 8))
    plt.axis('off')
    frames = [process_image(elem) for elem in samples_list]
    
    ani = animation.ArtistAnimation(fig, frames, interval=delay)  
    display(HTML(ani.to_html5_video()))

In [ ]:
#!g1.1
from start import init_train

Trainer = init_train("configs/StyleGAN.json", True, load_dataset=True)
Trainer.train_loop()

In [ ]:
#!g1.1
